### Код библиотеки для подготовки шаблонов

Ниже приведён код, аналогичный тому, который запускается для валидации шаблонов на нашей стороне.

Сейчас этот код представлен в виде ячейки, но позднее мы собираемся вынести его в отдельную библиотеку в рамках Датасферы.

In [1]:
import re
import tarfile
from typing import Tuple, List
from pathlib import Path
from typing import Optional


var_desc_regex = re.compile(r'\{.+?=.+?\}')
var_text_regex = re.compile(r'[^а-яёА-ЯЁ -]')
template_text_regex = re.compile(r'[^а-яёА-ЯЁ ,:.?!-]')
punct_regex = re.compile(r'[.?!]')


def validate_template(template: str, variable_sources: List[str]) -> Tuple[bool, str]:
    variable_sources = set(variable_sources)
    for var in var_desc_regex.findall(template):
        var_name, source_path = var[1:-1].split('=')
        if not source_path.endswith('.tsv'):
            return False, f'Описание переменной "{source_path}" имеет расширение, отличное от TSV'
        if source_path not in variable_sources:
            return False, f'Описание переменной "{source_path}" не было найдено'

    template = var_desc_regex.sub('', template).strip()    
    if template and punct_regex.findall(template[:-1]):
        return False, f'В середине шаблона присутствуют знаки конца предложения'
    
    max_len = 120
    if len(template) > max_len:
        return False, f'Длина шаблона без учёта вставок переменных не должна превышать {max_len} символов'

    errors = set(template_text_regex.findall(template))
    if errors:
        return False, f'Шаблон содержит недопустимые символы: {errors}'

    return True, ''


def validate_variable(variable: str) -> Tuple[bool, str]:
    max_len = 40
    if len(variable) > max_len:
        return False, f'Длина переменной не должна превышать {max_len} символов'

    errors = set(var_text_regex.findall(variable))
    if errors:
        return False, f'Переменная содержит недопустимые символы: {errors}'
    return True, ''


def check_templates(templates_path: str, variables_path: Optional[str]):
    templates_path = Path(templates_path)
    if not templates_path.exists():
        raise ValueError(f'\nФайл с шаблонами "{templates_path}" не найден')
    if not templates_path.name.endswith('.tsv'):
        raise ValueError(f'\nОписание шаблонов "{templates_path}" имеет расширение, отличное от TSV')

    if variables_path:
        variables_path = Path(variables_path)
        if not variables_path.exists() or not variables_path.is_dir():
            raise ValueError(f'\nДиректория "{variables_path}" не найдена')
        variable_sources = [f.name for f in variables_path.iterdir()]
    else:
        variable_sources = []

    for source in variable_sources:
        if not source.endswith('.tsv'):
            continue
        if not Path(variables_path, source).is_file():
            raise ValueError(f'\nОписание переменной "{source}" не найдено')
        for variable in Path(variables_path, source).read_text().splitlines():
            valid, msg = validate_variable(variable)
            if not valid:
                raise ValueError(f'\nНекорректная переменная "{variable}"\n{msg}')

    for template in templates_path.read_text().splitlines():
        valid, msg = validate_template(template, variable_sources)
        if not valid:
            raise ValueError(f'\nНекорректный шаблон "{template}"\n{msg}')


def prepare_stt_templates(templates_path: str, variables_path: Optional[str], output_path: str = 'templates.tar.gz'):
    check_templates(templates_path, variables_path)

    with tarfile.open(output_path, mode='w:gz') as tar:
        tar.add(templates_path, arcname='templates.tsv')
        if variables_path is not None:
            for source_path in Path(variables_path).iterdir():
                if not source_path.name.endswith('.tsv'):
                    continue
                tar.add(str(source_path), arcname=f'variables/{source_path.name}')

    print(f'Шаблоны успешно проверены и помещены в архив {output_path}.\n'
          'Теперь вы можете загрузить этот архив в датасферу, воспользовавшись соответствующим magic\'ом')

### Подготовка шаблонов

Подготовка шаблонов состоит из двух этапов.

#### Формирование шаблонов

В первую очередь, требуется сформировать список шаблонов (оформляется в виде одного файла, каждая строка в котором представляет собой отдельный шаблон), а также список переменных, которые будут подставляться в эти шаблоны (каждая переменная описывается отдельным файлом). Из текущих ограничей &mdash; шаблоны могут содержать лишь кириллицу и простые знаки препинания, переменные могут содержать лишь кириллицу и знаки дефисов, а также описания всех переменных должны лежать в отдельной папке.

В папке `templates` приведёны примеры того, как должны выглядеть шаблоны и переменные.

#### Загрузка шаблонов

После того, как список шаблонов и переменных будет сформирован, требуется загрузить их к нам. Для этого требуется:
1. С помощью функции `prepare_stt_templates` проверить шаблоны на корректность и сформировать архив данными для загрузки
2. Загрузить этот архив к нам с помощью magic'a `import_stt_templates`

Ниже приведён пример, как это сделать. Так, в функцию `prepare_stt_templates` требуется передать пути до шаблонов и до папки с переменными, а в magic `import_stt_templates` требуется передать путь до архива с шаблонами, кастомное название датасета и путь до файла, в который будет записан результат проверки шаблонов разработчиками.

In [2]:
prepare_stt_templates(templates_path='templates/templates.tsv', variables_path='templates/variables', output_path='templates.tar.gz')

Шаблоны успешно проверены и помещены в архив templates.tar.gz.
Теперь вы можете загрузить этот архив в датасферу, воспользовавшись соответствующим magic'ом


In [ ]:
#!nirvana
import_stt_templates --templates-archive voice_recorder_templates/templates.tar.gz --dataset-name fio --report-path voice_recorder_templates/report.txt

Starting workflow function task and uploading data...Starting workflow...
  56.3% [=============================>                     ]